#Configurações da Plataforma

In [263]:
import sys
import platform
import psutil
import subprocess


print("CPU: Intel(R) Xeon(R) CPU @ 2.20GHz\n"
      f"RAM: 12.68 GB\n"
      f"Disco: 107.72 GB\n"
      f"Versão do compilador: {sys.version}\n"
      f"Sistema operacional: {sys.platform}")

CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
RAM: 12.68 GB
Disco: 107.72 GB
Versão do compilador: 3.8.16 (default, Dec  7 2022, 01:12:13) 
[GCC 7.5.0]
Sistema operacional: linux


#Implementação dos Algoritmos

In [264]:
%%writefile main.cpp

#include <iostream>
#include <fstream>
#include <string>
#include <chrono>
#include <vector>
#include <bits/stdc++.h>
using namespace std;

int b[128];
const int ALPHABET_SIZE = 256;
static long MAXCHAR = 256;

int bruteForce(string pattern, string text) {
    for(int i = 0; i < text.length() - pattern.length() + 1; i++) {
        int j = 0;
        while (j < pattern.length() && text[i+j] == pattern[j]) {
            j += 1;
        }
        if (j == pattern.length()) {
            j = 0;
            //cout<<"FB Casou em "<<i<<endl;
        }
    }
}

void shiftAnd(string text, string pattern){
    vector<int> masks;
    int state;
    for(int i = 0; i < 256; i++){
        masks.push_back(0);
    }
    
    for(int i = 0; i < pattern.size(); i++){
        masks[__toascii(pattern[i])] |= 1 << i;
    }
    
    state = 0;
    
    for(int i = 0; i < text.size(); i++){
        state = ((state << 1) | 1) & masks[__toascii(text[i])];
        if(state & (1 << (pattern.size()-1))){
            //cout<<"SE: Casou em "<<i - pattern.size() + 2<<endl;
            continue;
        }
    }
}

void BMH(string T,string P){
    long n = T.length();
    long m = P.length();

    long i, j, k, d[MAXCHAR + 1];
    for(j=0;j<=MAXCHAR; j++)d[j] = m;
    for(j=1;j<m;j++)d[P[j-1]] = m-j;
    i=m;
    while(i<=n){
        k=i;
        j=m;
        while(T[k-1]==P[j-1] && j>0){
            k--;j--;
        }
        if(j==0) /*cout<<"BMH Casou em "<<k+1<<endl*/; 
        i+=d[T[i-1]];
    }
}

void BMHS(string T, string P){
    long n = T.length();
    long m = P.length();

    long i, j, k, d[MAXCHAR + 1];
    for(j=0;j<=MAXCHAR;j++)d[j]=m+1;
    for(j=1;j<=m;j++)d[P[j-1]]=m-j+1;
    i=m;
    while(i<=n){
        k=i;
        j=m;
        while(T[k-1]==P[j-1] && j>0){k--;j--;}
        if(j==0)
        /*cout<<"BMHS Casou em "<<k+1<<endl*/;
        i+=d[T[i]];
    }
}

void shiftAndApprox(string T, string P, long k){
    long n = T.length();
    long m = P.length();

    long Masc[MAXCHAR],i,j,Ri,Rant,Rnovo;
    long R[2+1];
    for(i=0;i<MAXCHAR;i++)Masc[i]=0;
    for(i=1;i<=m;i++){Masc[P[i-1] + 127] |= 1 << (m-i);}
    R[0] = 0; Ri = 1 << (m-1);
    for(j=1;j<=k;j++)R[j]=(1 << (m-j)) | R[j-1];
    for(i=0;i<n;i++){
        Rant= R[0];
        Rnovo = ((((unsigned long)Rant) >> 1) | Ri) & Masc[T[i] + 127];
        R[0] = Rnovo;
        for(j=1;j<=k;j++){
            Rnovo=((((unsigned long)R[j]) >> 1) & Masc[T[i]+127]) | Rant |(((unsigned long)(Rant | Rnovo)) >> 1 );
            Rant= R[j];R[j]= Rnovo | Ri;
        }
        if((Rnovo & 1) != 0) /*cout<<"SAA " << k <<" Casou em "<<i<<endl*/;
    }
}

int main() {
    // LENDO ARQUIVOS DE TEXTO
    ifstream file("T_500.txt");
    string T500;
    T500 = string((istreambuf_iterator<char>(file)), istreambuf_iterator<char>());
    file.close();

    ifstream file2("T_1000.txt");
    string T1000;
    T1000 = string((istreambuf_iterator<char>(file2)), istreambuf_iterator<char>());
    file2.close();

    ifstream file3("T_1500.txt");
    string T1500;
    T1500 = string((istreambuf_iterator<char>(file3)), istreambuf_iterator<char>());
    file3.close();

    ifstream file4("T_2000.txt");
    string T2000;
    T2000 = string((istreambuf_iterator<char>(file4)), istreambuf_iterator<char>());
    file4.close();

    ifstream file5("T_5000.txt");
    string T5000;
    T5000 = string((istreambuf_iterator<char>(file5)), istreambuf_iterator<char>());
    file5.close();

    // CRIA VETOR COM OS TEXTOS
    string textos[5] = {T500, T1000, T1500, T2000, T5000};

    vector<double> brutal_force, bmh, bmhs, shift_and, shift_and_aprox_k1, shift_and_aprox_k2;
    for(int i = 0; i < 5; i++){
      brutal_force.push_back(0);
      bmh.push_back(0);
      bmhs.push_back(0);
      shift_and.push_back(0);
      shift_and_aprox_k1.push_back(0);
      shift_and_aprox_k2.push_back(0);
    }

    ofstream myfile("Output.csv");
    myfile << "alg,T500,T1000,T1500,T2000,T5000\n";

    std::chrono::time_point<std::chrono::system_clock> start, end;
    std::chrono::duration<double> elapsed_seconds;
    string P = "dolor";

    for(int k = 0; k < 10; k++){
        for(int i = 0; i < 5; i++){
          start = std::chrono::system_clock::now();
          bruteForce(P, textos[i]); ////////////////
          end = std::chrono::system_clock::now();
          elapsed_seconds = end - start;
          std::time_t end_time = std::chrono::system_clock::to_time_t(end);
          /*std::cout << "finished computation at " << std::ctime(&end_time)
                    << "elapsed time: " << chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count() << "s\n";*/
          brutal_force[i] += (chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count());

          start = std::chrono::system_clock::now();
          BMH(textos[i], P); /////////////////////
          end = std::chrono::system_clock::now();
          elapsed_seconds = end - start;
          end_time = std::chrono::system_clock::to_time_t(end);
          bmh[i] += (chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count());

          start = std::chrono::system_clock::now();
          BMHS(textos[i], P); /////////////////////
          end = std::chrono::system_clock::now();
          elapsed_seconds = end - start;
          end_time = std::chrono::system_clock::to_time_t(end);
          bmhs[i] += (chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count());

          start = std::chrono::system_clock::now();
          shiftAnd(textos[i], P); /////////////////////
          end = std::chrono::system_clock::now();
          elapsed_seconds = end - start;
          end_time = std::chrono::system_clock::to_time_t(end);
          shift_and[i] += (chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count());

          start = std::chrono::system_clock::now();
          shiftAndApprox(textos[i], P, 1); /////////////////////
          end = std::chrono::system_clock::now();
          elapsed_seconds = end - start;
          end_time = std::chrono::system_clock::to_time_t(end);
          shift_and_aprox_k1[i] += (chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count());

          start = std::chrono::system_clock::now();
          shiftAndApprox(textos[i], P, 2); /////////////////////
          end = std::chrono::system_clock::now();
          elapsed_seconds = end - start;
          end_time = std::chrono::system_clock::to_time_t(end);
          shift_and_aprox_k2[i] += (chrono::duration_cast<chrono::nanoseconds>(elapsed_seconds).count());
        }
    }

    for(int i = 0; i < 5; i++){
      brutal_force[i] = brutal_force[i]/10;
      bmh[i] = bmh[i]/10;
      bmhs[i] = bmhs[i]/10;
      shift_and[i] = shift_and[i]/10;
      shift_and_aprox_k1[i] = shift_and_aprox_k1[i]/10;
      shift_and_aprox_k2[i] = shift_and_aprox_k2[i]/10;
    }

    // ============================= FORÇA BRUTA ======================================  
    myfile << "Brute Force";
    for(int i = 0; i < 5; i++){
      myfile << "," << brutal_force[i];
      cout << brutal_force[i] << " | ";
    }
    myfile << "\n";

    cout << '\n';
    
    // ============================== BMH ====================================== 
    myfile << "BMH";
    for(int i = 0; i < 5; i++){
      myfile << "," << bmh[i];
      cout << bmh[i] << " | ";
    }
    myfile << "\n";

    cout << '\n';
    
    // ============================== BMHS ====================================== 
    myfile << "BMHS";
    for(int i = 0; i < 5; i++){
      myfile << "," << bmhs[i];
      cout << bmhs[i] << " | ";
    }
    myfile << "\n";

    cout << '\n';
    
    // ============================== SHIFT AND ====================================== 
    myfile << "Shift And";
    for(int i = 0; i < 5; i++){
      myfile << "," << shift_and[i];
      cout << shift_and[i] << " | ";
    }
    myfile << "\n";

    cout << '\n';
    
    // ============================== SHIFT AND APROX K1 ======================================
    myfile << "Shift And k1";
    for(int i = 0; i < 5; i++){
      myfile << "," << shift_and_aprox_k1[i];
      cout << shift_and_aprox_k1[i] << " | ";
    }
    myfile << "\n";

    cout << '\n';

    // ============================== SHIFT AND APROX K2 ======================================
    myfile << "Shift And k2";
    for(int i = 0; i < 5; i++){
      myfile << "," << shift_and_aprox_k2[i];
      cout << shift_and_aprox_k2[i] << " | ";
    }
    myfile << "\n";

    myfile.close();
    return 0;
}

Overwriting main.cpp


In [265]:
%%shell

g++ main.cpp -o main
./test

9239.8 | 20744.1 | 26063.3 | 35403.9 | 87150 | 
1452.9 | 2393.7 | 3377 | 4345.7 | 14119.9 | 
1389.8 | 2301 | 3111.9 | 4051.4 | 10871.1 | 
10729.1 | 15381.8 | 20288.9 | 25282.9 | 55632.8 | 
4089.2 | 7755 | 13901.8 | 15133.1 | 40078.1 | 
6012.9 | 11920.2 | 17249.1 | 26261.8 | 56639.2 | 

#Plotagem dos Gráficos e Tabelas

In [266]:
import pandas as pd
import plotly.graph_objects as go

In [267]:
df = pd.read_csv('Output.csv')
means = []
for i in df.iterrows():
    means.append(i[1][1:6].tolist())

labels = ['500','1000','1500','2000','5000']
columns = ['Brute Force','BMH','BMHS','S.A Exactly','S.A Approx K=1','S.A Approx K=2']

In [268]:
graph = go.Figure()
graph.update_layout(
    title="Algoritmos de Casamento de Padrão",
    xaxis_title="Quantidade de Caracteres",
    yaxis_title="Tempo(ns)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    ),
  width=1400, 
  height=800
)

for i in range(6):
  graph.add_trace(go.Scatter(x=labels, y=means[i],mode='lines',name=columns[i]))

graph.show()

In [269]:
labels = ['<b>Algoritmos</b>','<b>T500</b>','<b>T1000</b>','<b>T1500</b>','<b>T2000</b>','<b>T5000</b>']
means_transpose = list(np.transpose(means))
means_transpose.insert(0, columns)

headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

table = go.Figure(data=[go.Table(header=dict(values=labels,
                                             line_color='darkslategray',
                                             fill_color=headerColor,
                                             font=dict(color='white', size=12)),
                 cells=dict(values=means_transpose,
                            line_color='darkslategray',
                            fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor,rowEvenColor]*5],
                            font=dict(color='darkslategray', size=12)))])

table.update_layout(
  width=1400, 
  height=800
)


table.show()

In [270]:
labels = ['<b>T500</b>','<b>T1000</b>','<b>T1500</b>','<b>T2000</b>','<b>T5000</b>']

bar = go.Figure(data=[
    go.Bar(name='Brute Force', x=labels, y=means[0]),
    go.Bar(name='BMH', x=labels, y=means[1]),
    go.Bar(name='BMHS', x=labels, y=means[2]),
    go.Bar(name='S.A Exactly', x=labels, y=means[3]),
    go.Bar(name='S.A Approx K=1', x=labels, y=means[4]),
    go.Bar(name='S.A Approx K=2', x=labels, y=means[5]),
])

# Change the bar mode
bar.update_layout(
    barmode='group',
    title="Algoritmos de Casamento de Padrão",
    xaxis_title="Quantidade de Caracteres",
    yaxis_title="Tempo(ns)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

bar.show()